In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data=pd.read_excel('result.xlsx')[0:32]
data

In [ ]:
X=data.iloc[:,1:4]

In [ ]:
X=np.array(X)

In [ ]:
len(X)

In [ ]:
y=data[['phase/ 0 alpha/1 beta/2 mix']]

In [ ]:
y=np.array(y).ravel()

In [ ]:
y

In [ ]:
from sklearn.model_selection import KFold
import torch
import torch.nn as nn
import torch.optim as optim
from scipy.stats import entropy
import torchbnn as bnn
from sklearn.preprocessing import MinMaxScaler

In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
   


kf = KFold(n_splits=5, shuffle=True, random_state=42)
cv_accuracies = []

for train_index, val_index in kf.split(X):
   
    X_train_np, X_val_np = X[train_index], X[val_index]
    y_train_np, y_val_np = y[train_index], y[val_index]

    
    scaler = MinMaxScaler(feature_range=(0, 1))  
    X_train_scaled = scaler.fit_transform(X_train_np)
    X_val_scaled   = scaler.transform(X_val_np)

   
    X_train_t = torch.tensor(X_train_scaled, dtype=torch.float32, device=device)
    y_train_t = torch.tensor(y_train_np, dtype=torch.long, device=device)
    X_val_t   = torch.tensor(X_val_scaled, dtype=torch.float32, device=device)
    y_val_t   = torch.tensor(y_val_np, dtype=torch.long, device=device)

    
    model = nn.Sequential(
        bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=X_train_t.shape[1], out_features=50),
        nn.ReLU(),
        bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=50, out_features=len(np.unique(y)))
    ).to(device)

    criterion = nn.CrossEntropyLoss()
    kl_loss = bnn.BKLLoss(reduction='mean')
    optimizer = optim.Adam(model.parameters(), lr=1e-3)

    for epoch in range(500):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train_t)
        loss = criterion(outputs, y_train_t) + 0.1 * kl_loss(model)
        loss.backward()
        optimizer.step()

  
    model.eval()
    with torch.no_grad():
        val_preds = model(X_val_t).argmax(dim=1)
        acc = (val_preds == y_val_t).float().mean().item()
        
        cv_accuracies.append(acc)

mean_acc = np.mean(cv_accuracies)
std_cv_acc = np.std(cv_accuracies)  
sem_cv_acc = std_cv_acc / np.sqrt(len(cv_accuracies))

print("Mean Accuracy:", mean_acc)
print("Accuracy SEM:", sem_cv_acc)
